In [2]:
import seaborn as sns
import pandas as pd
from pyspark.sql import SparkSession

from sklearn.datasets import load_breast_cancer

spark = SparkSession.builder.appName('clasificacion_binaria').getOrCreate()
data = load_breast_cancer()
df = pd.DataFrame(data.data, columns=data.feature_names)
df['label'] = data.target

# Quitamos espacio por _
df.columns = [col.replace(' ', '_') for col in df.columns]

df.head(3)

,mean_radius,mean_texture,mean_perimeter,mean_area,mean_smoothness,mean_compactness,mean_concavity,mean_concave_points,mean_symmetry,mean_fractal_dimension,...,worst_texture,worst_perimeter,worst_area,worst_smoothness,worst_compactness,worst_concavity,worst_concave_points,worst_symmetry,worst_fractal_dimension,label
0,17.99,10.38,122.8,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.6,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.9,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.8,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.0,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.5,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0


In [3]:
# creamos el dataframe de spark
df = spark.createDataFrame(df)

In [ ]:
# Seleecionamos todas las columnas excepto 'label'
feature_cols = [col for col in df.columns if col != 'label']
feature_cols

['mean_radius',
 'mean_texture',
 'mean_perimeter',
 'mean_area',
 'mean_smoothness',
 'mean_compactness',
 'mean_concavity',
 'mean_concave_points',
 'mean_symmetry',
 'mean_fractal_dimension',
 'radius_error',
 'texture_error',
 'perimeter_error',
 'area_error',
 'smoothness_error',
 'compactness_error',
 'concavity_error',
 'concave_points_error',
 'symmetry_error',
 'fractal_dimension_error',
 'worst_radius',
 'worst_texture',
 'worst_perimeter',
 'worst_area',
 'worst_smoothness',
 'worst_compactness',
 'worst_concavity',
 'worst_concave_points',
 'worst_symmetry',
 'worst_fractal_dimension']

In [ ]:

from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(
    inputCols=feature_cols,
    outputCol='features'
)

df_asembled = assembler.transform(df).select('features', 'label') # Transformamos y seleccionamos las columnas de una vez
df_asembled.show(2)

+--------------------+-----+
|            features|label|
+--------------------+-----+
|[17.99,10.38,122....|    0|
|[20.57,17.77,132....|    0|
+--------------------+-----+
only showing top 2 rows



In [ ]:
# Particionamos los datos para entrenamiento y test
df_train, df_test = df_asembled.randomSplit([0.8, 0.2], seed=42)

In [8]:
from pyspark.ml.classification import LogisticRegression, DecisionTreeClassifier, RandomForestClassifier, GBTClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator


lr = LogisticRegression() # Modelo
model = lr.fit(df_train) # Entrenamiento
df_pred = model.transform(df_test) # Prediccion
df_pred.show(3)


+--------------------+-----+--------------------+-----------+----------+
|            features|label|       rawPrediction|probability|prediction|
+--------------------+-----+--------------------+-----------+----------+
|[8.618,11.79,54.3...|    1|[-1633.8350744086...|  [0.0,1.0]|       1.0|
|[9.504,12.44,60.3...|    1|[-1684.3539217250...|  [0.0,1.0]|       1.0|
|[10.95,21.35,71.9...|    0|[873.702083751711...|  [1.0,0.0]|       0.0|
+--------------------+-----+--------------------+-----------+----------+
only showing top 3 rows



In [11]:
# Calculo de las metricas

evaluator_auc = BinaryClassificationEvaluator(metricName='areaUnderROC')
evaluator_accuracy = MulticlassClassificationEvaluator(metricName='accuracy')
evaluator_f1 = MulticlassClassificationEvaluator(metricName='f1')
evaluator_precision = MulticlassClassificationEvaluator(metricName='weightedPrecision')
evaluator_recall = MulticlassClassificationEvaluator(metricName='weightedRecall')                                                  
 
 
print('auc', evaluator_auc.evaluate(df_pred))                                                        
print('accuracy', evaluator_accuracy.evaluate(df_pred))
print('f1', evaluator_f1.evaluate(df_pred))
print('precision', evaluator_precision.evaluate(df_pred))
print('weightedRecall', evaluator_recall.evaluate(df_pred))


auc 0.9932026143790849
accuracy 0.9523809523809523
f1 0.9523809523809523
precision 0.9523809523809523
weightedRecall 0.9523809523809523


In [12]:
tree = DecisionTreeClassifier()
model = tree.fit(df_train)
df_pred = model.transform(df_test)

print('auc', evaluator_auc.evaluate(df_pred))                                                        
print('accuracy', evaluator_accuracy.evaluate(df_pred))
print('f1', evaluator_f1.evaluate(df_pred))
print('precision', evaluator_precision.evaluate(df_pred))
print('weightedRecall', evaluator_recall.evaluate(df_pred))


auc 0.9351633986928104
accuracy 0.9365079365079365
f1 0.9366894366894365
precision 0.9373530751147707
weightedRecall 0.9365079365079365
